In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import pandas as pd

In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from torch import nn
from transformers import AutoTokenizer, AutoModel
from torch import nn
from sklearn.metrics import f1_score, roc_auc_score
from tqdm import tqdm
import os

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cuda'

In [3]:
from sklearn.model_selection import train_test_split

In [5]:
df = pd.read_csv("/kaggle/input/vkstazhka2024/train_spam.csv")
df.head(5)

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...


In [6]:
df["text_type"].value_counts()

text_type
ham     11469
spam     4809
Name: count, dtype: int64

In [7]:
mean_weight = len(df) / 2

In [8]:
cat_weights = torch.tensor([mean_weight / 11469, mean_weight / 4809], device=device)

In [9]:
categories = {"ham": 0, "spam": 1}

In [10]:
df["text_type"] = df["text_type"].apply(lambda x: categories[x])
df.head(5)

,text_type,text
0,0,make sure alex knows his birthday is over in f...
1,0,a resume for john lavorato thanks vince i will...
2,1,plzz visit my website moviesgodml to get all m...
3,1,urgent your mobile number has been awarded wit...
4,0,overview of hr associates analyst project per ...


In [20]:
df.head()

,text_type,text
0,0,make sure alex knows his birthday is over in f...
1,0,a resume for john lavorato thanks vince i will...
2,1,plzz visit my website moviesgodml to get all m...
3,1,urgent your mobile number has been awarded wit...
4,0,overview of hr associates analyst project per ...


In [21]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [40]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [41]:
text = df["text"].values
labels = torch.tensor(df["text_type"].values)

In [42]:
features = tokenizer(text.tolist(), padding=True, truncation=True, return_tensors='pt')
features
input_ids = features["input_ids"]
attention_mask = features["attention_mask"]

In [43]:
dataset = TensorDataset(input_ids.to(device), attention_mask.to(device), labels.to(device))

In [44]:
train_dataset, test_dataset = train_test_split(dataset, random_state=42, test_size=0.35)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
test_dataloader = DataLoader(test_dataset, batch_size=16)

In [45]:
class RubertTinyClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.bert = AutoModel.from_pretrained("google-bert/bert-base-uncased").to(device)
        for param in self.bert.parameters():
            param.requires_grad = False

        self.pre_trained_output_len = self.bert.encoder.layer[1].output.dense.out_features
        self.linear = torch.nn.Linear(self.pre_trained_output_len, num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids=None):
        return self.linear(self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).pooler_output)

In [46]:
model = RubertTinyClassifier(2).to(device)
loss_fn = nn.CrossEntropyLoss(cat_weights)

In [47]:
optim = torch.optim.Adam(model.parameters(), lr=0.00001)

In [48]:
def accuracy_score(y_pred, y_true):
    return (y_pred.argmax(axis=1) == y_true).sum() / len(y_pred)

In [49]:
from torch.nn.functional import softmax

In [50]:
ids, mask, label = next(iter(test_dataloader))
with torch.inference_mode():
    pred = model(ids, mask)

accuracy_score(pred, label)

tensor(0.6250, device='cuda:0')

In [51]:
def train(model, dataloader, loss_fn, optim, epochs=10):
    os.makedirs('checkpoint', exist_ok=True)
    model.train()
    for epoch in range(epochs):
        mean_loss = 0
        mean_score = 0
        micro = 0
        for input_ids, attention_mask, label in tqdm(dataloader):
            model.zero_grad()
            
            pred = model(input_ids, attention_mask)
            
            loss = loss_fn(pred, label)
            
            mean_loss += loss.item()
            
            mean_score += accuracy_score(pred, label)
            
            loss.backward()
            
            optim.step()
        
        torch.save(model.state_dict(), f'checkpoint/BertClassifier{epoch}.pt')
        
        mean_loss /= len(dataloader)
        mean_score /= len(dataloader)
        micro /= len(dataloader)
        mean_score *= 100
        
        test_result = test(model, test_dataloader, loss_fn)
        print(f"Epoch: {epoch} | Train Loss: {mean_loss:.3f} | micro-f1-score: {mean_score:.3f} |" + test_result)
        

In [52]:
def test(model, dataloader, loss_fn):
    model.eval()
    mean_loss = 0
    mean_score = 0
    micro = 0
    rc = 0
    with torch.inference_mode():
        for input_ids, attention_mask, label in tqdm(dataloader):

            pred = model(input_ids, attention_mask)

            loss = loss_fn(pred, label)

            mean_loss += loss.item()

            mean_score += accuracy_score(pred, label)
            
            rc += roc_auc_score(pred, label)
            

        mean_loss /= len(dataloader)
        mean_score /= len(dataloader)
        mean_score *= 100
        rc /= len(dataloader) * 100
        result = f"| Test Loss: {mean_loss:.3f} | micro-f1-score: {mean_score:.3f} | AUROC: {rc}"
    model.train()
    return result

In [53]:
train(model, train_dataloader, loss_fn, optim, epochs=60)

  0%|          | 0/357 [00:00<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [152]:
test(model, test_dataloader, loss_fn)

 43%|████▎     | 155/357 [00:42<00:55,  3.61it/s]/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
100%|██████████| 357/357 [01:38<00:00,  3.62it/s]


'| Test Loss: 0.235 | micro-f1-score: 91.474 | AUROC: 0.009622547775506973'

In [153]:
from IPython.display import FileLink

In [154]:
FileLink("./checkpoint/BertClassifier1.pt")

/kaggle/working/checkpoint/BertClassifier1.pt

In [ ]:
!pwd

In [ ]:
model = RubertTinyClassifier(10)
model.load_state_dict(torch.load("/kaggle/working/checkpoint/BertClassifier52.pt"))


In [24]:
modelbert = torch.load("/kaggle/input/tunedbert/pytorch/tunedbertvk/1/BertClassifier0(1).pt")

In [150]:


ids, mask, label = next(iter(test_dataloader))
with torch.inference_mode():
    pred = model(ids, mask)

metric(pred, label)

tensor(0.9667, device='cuda:0')

In [22]:
df

,text_type,text
0,0,make sure alex knows his birthday is over in f...
1,0,a resume for john lavorato thanks vince i will...
2,1,plzz visit my website moviesgodml to get all m...
3,1,urgent your mobile number has been awarded wit...
4,0,overview of hr associates analyst project per ...
...,...,...
16273,1,if you are interested in binary options tradin...
16274,1,dirty pictureblyk on aircel thanks you for bei...
16275,0,or you could do this g on mon 1635465 sep 1635...
16276,0,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [11]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np

vectorizer = CountVectorizer(binary=True, max_features=1000)
X = vectorizer.fit_transform(df['text'])
y = df['text_type']

cv = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
roc_auc_scores = []

for train_index, val_index in cv.split(X, y):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    val_predictions = model.predict_proba(X_val)[:, 1]  
    roc_auc = roc_auc_score(y_val, val_predictions)
    roc_auc_scores.append(roc_auc)

average_roc_auc = np.mean(roc_auc_scores)
print(f"Средний ROC AUC по кросс-валидации: {average_roc_auc:.2f}")


Средний ROC AUC по кросс-валидации: 0.97


In [13]:
test = pd.read_csv('/kaggle/input/vkstazhka2024/test_spam.csv')

In [14]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

X_test = vectorizer.transform(test['text'])

test_predictions = model.predict(X_test)

test['text_type'] = test_predictions

test.to_csv('test_predictions.csv', index=False)

print("Предсказания сохранены в файл 'test_predictions.csv'.")


Предсказания сохранены в файл 'test_predictions.csv'.


NameError: name 'test' is not defined